
# 0 - Imports and defining functions

In [43]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

import os

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 1 - Data loading

In [44]:
# Input folder name
inputFolder = 'PAIR_002'
# Concatenate folder name with the path
inputPath = '../data/' + inputFolder + '/'
mesh1 = TriMesh(inputPath + 'target.off', area_normalize=True, center=False)
mesh2 = TriMesh(inputPath + 'source.off', area_normalize=True, center=False)

# 2- Visualization of eigen functions

In [45]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=100, intrinsic=False, verbose=True);

# set up a figure twice as wide as it is tall
i = 0
d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

Computing 100 eigenvectors
	Done in 5.10 s


# 3 - Computing the functional map

**Displaying data**

In [46]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])
mp.subplot(mesh2.vertlist, mesh2.facelist, None, s=[2, 2, 1], data=d)

Mesh 1 : 32306 vertices, 64740 faces
Mesh 2 : 32065 vertices, 64170 faces


**Add correspondences**

In [47]:
loadLandmark = True
if loadLandmark:
    landmarks = np.loadtxt(inputPath +'landmarks.txt',dtype=int)[:10]  # loading N landmarks
    display(landmarks)

    landmarks_mesh1 = TriMesh(mesh1.vertlist[landmarks[:,0]])
    landmarks_mesh2 = TriMesh(mesh2.vertlist[landmarks[:,1]])

    print('Retrieved matching faces from CT & RGBD pointclouds')
    
else:
     landmarks = []

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)

if loadLandmark:
    color = c=np.random.rand(*landmarks_mesh1.vertlist.shape)
    p1.add_points(landmarks_mesh1.vertlist, c = color, shading={"point_size": 0.3})
    p2.add_points(landmarks_mesh2.vertlist, c = color, shading={"point_size": 0.3})


array([[27358, 27505],
       [31694, 28360],
       [12643,  9898],
       [25320, 27680],
       [22629, 18965],
       [14273, 19124]])

Retrieved matching faces from CT & RGBD pointclouds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.096350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.114139…

**Generate functional map directly from landmarks**

In [66]:
# Initialize vector 1 to 5
p2p_21 = np.array([0,1,2,3,4,5])
#p2p_21 = landmarks[:,1]
dim = len(landmarks[:,0])

resultingFM = FunctionalMapping(mesh1=mesh1, mesh2=mesh2)
resultingFM.mesh1.process(k=50, intrinsic=False, verbose=True);
resultingFM.mesh2.process(k=50, intrinsic=False, verbose=True);


print(f'p2p map  : {p2p_21} \n')
print(f'eigen vectors 1 : {resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim]} \n')
print(f'eigen vectors 2 : {resultingFM.mesh2.eigenvectors[landmarks[:,1], :dim]} \n')
print(f'dim : {dim} \n') 

#test = pyFM.spectral.convert.p2p_to_FM(p2p_21, mesh1, mesh2, use_adj=False, subsample=None, use_ANN=False, n_jobs=1)
import pyFM.spectral.convert
#test = pyFM.spectral.convert.mesh_p2p_to_FM(p2p_21, mesh1, mesh2, dim, [landmarks[:,0], landmarks[:,1]])
test = pyFM.spectral.convert.p2p_to_FM(p2p_21, resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim], resultingFM.mesh2.eigenvectors[landmarks[:,1], :])
print(f'FM : {test} \n')

# Resuling FM
resultingFM.FM = test
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)



Computing 50 eigenvectors
	Done in 2.00 s
p2p map  : [0 1 2 3 4 5] 

eigen vectors 1 : [[-1.          1.48968982  0.89492381 -1.15140567  2.80914176  0.99376185]
 [-1.          0.61961321 -0.34264158 -0.45098227  0.73704563 -2.48844354]
 [-1.          0.12337274 -1.55119641  2.20696165  0.63055728  1.83645826]
 [-1.         -0.50365086 -0.32307601  0.08005977 -0.50859054 -1.82777239]
 [-1.         -1.89376252  1.36951647  0.31245958  0.45575513  0.07065674]
 [-1.          0.03774584 -0.29230713 -1.37260114 -1.22989819  1.17721129]] 

eigen vectors 2 : [[-1.         -1.8358493  -1.80731605  0.64012196  0.09971144 -0.97749885]
 [-1.         -0.60471167  0.2211168  -0.25272809 -1.49404041 -2.31632836]
 [-1.         -0.07910433  1.61538981  2.39411024 -0.18793126  0.14141135]
 [-1.          0.53406234  0.41705964 -0.33048053  2.41587613  1.53251022]
 [-1.          2.03070688 -1.42377698  0.46799172  1.753805   -2.45098212]
 [-1.         -0.04670937  0.30640095 -1.55473369 -0.35136768  0.29

AttributeError: 'numpy.ndarray' object has no attribute 'FM'

In [ ]:
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=12, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=12, intrinsic=False, verbose=True);

p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=9, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=9, intrinsic=False, verbose=True);

p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [65]:
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=9, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=9, intrinsic=False, verbose=True);



resultingFM.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
resultingFM._FM_base = test

resultingFM.zoomout_refine(nit=5, step = 5, verbose=True)
print(resultingFM.FM.shape)

p2p_21_zo = resultingFM.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

AssertionError: Not enough eigenvectors on target :         75 are needed when 50 are provided